In [1]:
import json
import urllib.request
import ssl
import platform
import time
import sys
import time
import pprint
import pandas as pd
from time import gmtime, strftime

#'http://novasoft-th.com/indoor'
url = 'http://localhost/indoor'
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

def bouncingSleep(retryCount):
    waittime = 8 if (retryCount[0])**2 > 8 else (retryCount[0])**2
    if((retryCount[0])**2 < 8):
        retryCount[0]+=1
    time.sleep(waittime)

def getJob():
    data = {
            'COMMAND': "REQUESTJOB",
            'NAME' : platform.node()
    }
    req = urllib.request.Request(url)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(data)
    jsondataasbytes = jsondata.encode('utf-8')

    retryCount = [0]
    while True:
        try:
            response = urllib.request.urlopen(req, jsondataasbytes,context=ctx)
            retryCount = [0]
        except:
            #sys.stdout.write('\rnumber is %d' %i)
            sys.stdout.write('\rRequesting job...')
            sys.stdout.flush()
            bouncingSleep(retryCount)
            continue   
        data = json.loads(response.read().decode('utf-8'))
        if data["RESPONSE"] == "ASSIGNED":            
            return data
        elif data["RESPONSE"] == "FINISHED":
            sys.stdout.write('\rNo more job to do. listening for any further...')
            sys.stdout.flush()
            
            bouncingSleep(retryCount)
            continue  
        elif data["RESPONSE"] == "WAIT":
            sys.stdout.write('\rWait')
            sys.stdout.flush()
            bouncingSleep(retryCount)
            continue         
        else:    
            bouncingSleep(retryCount)
            continue  
def submitResult(result):
    _data = {
            'COMMAND': "SUBMITRESULT",
    }
    data = {**_data , **result}
    req = urllib.request.Request(url)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(data)
    jsondataasbytes = jsondata.encode('utf-8')
    retryCount = [0]
    while True:
        try:
            response = urllib.request.urlopen(req, jsondataasbytes,context=ctx)
            retryCount = [0]
        except:
            retryCount += 1
            print("Submitting failed. Retry #" + str(retry))
            bouncingSleep(retryCount)
            continue   
        data = json.loads(response.read().decode('utf-8'))
        if "RESPONSE" in data:
            return data
        else:
            raise Exception("unknown response")

In [2]:
def paramsCleanup(params):
    params["sae_hidden_layers"] =  params["sae_hidden_layers"].replace("[","")
    params["sae_hidden_layers"] =  params["sae_hidden_layers"].replace("]","")
    params["sae_hidden_layers"] =  params["sae_hidden_layers"].replace(" ","")
    params["classifier_hidden_layers"] =  params["classifier_hidden_layers"].replace("[","")
    params["classifier_hidden_layers"] =  params["classifier_hidden_layers"].replace("]","")
    params["classifier_hidden_layers"] =  params["classifier_hidden_layers"].replace(" ","")


In [ ]:

while(True):
    params = getJob()
    paramsCleanup(params)
    
    
    print("parameters retrived " + strftime("[%Y-%m-%d %H:%M:%S]", gmtime()))
    print ('*' * 50)
    print(pd.DataFrame(list(params.items()), columns=['Parameter', 'Value']))
    print ('*' * 50)
    
    N = str(params["N"])
    acc_bf = str(params["acc_bf"])
    acc_bld = str(params["acc_bld"])
    acc_flr = str(params["acc_flr"])
    batch_size = str(params["batch_size"])
    classifier_activation= str(params["classifier_activation"])
    classifier_bias= str(params["classifier_bias"])
    classifier_hidden_layers= str(params["classifier_hidden_layers"])
    classifier_loss= str(params["classifier_loss"])
    classifier_optimizer= str(params["classifier_optimizer"])
    dropout= str(params["dropout"])
    epochs= str(params["epochs"])
    loc_failure= str(params["loc_failure"])
    mean_pos_err= str(params["mean_pos_err"])
    mean_pos_err_weighted= str(params["mean_pos_err_weighted"])
    random_seed= str(params["random_seed"])
    sae_activation= str(params["sae_activation"])
    sae_bias= str(params["sae_bias"])
    sae_hidden_layers= str(params["sae_hidden_layers"])
    sae_loss= str(params["sae_loss"])
    sae_optimizer= str(params["sae_optimizer"])
    scaling= str(params["scaling"])
    submitted_date= str(params["submitted_date"])
    time_spent= str(params["time_spent"])
    trained_by= str(params["trained_by"])
    training_ratio= str(params["training_ratio"])

    %run client.py -S $sae_hidden_layers -C $classifier_hidden_layers -D $dropout -N $N --scaling $scaling --batch_size $batch_size -R $random_seed -E $epochs -T $training_ratio 

    submit = submitResult(output)
    if submit["RESPONSE"] == "SUCCESS":            
        print("result submitted " + strftime("[%Y-%m-%d %H:%M:%S]", gmtime()))
    elif submit["RESPONSE"] == "FAIL":
        print("result is rejected " + strftime("[%Y-%m-%d %H:%M:%S]", gmtime()))

parameters retrived [2018-03-14 08:08:47]
**************************************************
                   Parameter                Value
0                random_seed                    0
1             training_ratio                  0.9
2                     epochs                   20
3                 batch_size                   10
4                          N                    8
5                    scaling                  0.2
6          sae_hidden_layers                   64
7             sae_activation                 relu
8                   sae_bias                FALSE
9              sae_optimizer                 adam
10                  sae_loss                  mse
11  classifier_hidden_layers               64,128
12     classifier_activation                 relu
13           classifier_bias                FALSE
14      classifier_optimizer                 adam
15           classifier_loss  binary_crossentropy
16                   dropout                  0.2
17     

Using TensorFlow backend.



Part 1: buidling an SAE encoder ...
Epoch 1/20
17928/17928 [==============================] - 83s 5ms/step - loss: 0.4345
Epoch 2/20
17928/17928 [==============================] - 4s 201us/step - loss: 0.3670
Epoch 3/20
17928/17928 [==============================] - 4s 202us/step - loss: 0.3607
Epoch 4/20
17928/17928 [==============================] - 4s 200us/step - loss: 0.3589
Epoch 5/20
17928/17928 [==============================] - 4s 198us/step - loss: 0.3581
Epoch 6/20
17928/17928 [==============================] - 4s 208us/step - loss: 0.3576
Epoch 7/20
17928/17928 [==============================] - 4s 208us/step - loss: 0.3574
Epoch 8/20
17928/17928 [==============================] - 4s 207us/step - loss: 0.3570
Epoch 9/20
17928/17928 [==============================] - 4s 202us/step - loss: 0.3569
Epoch 10/20
17928/17928 [==============================] - 4s 198us/step - loss: 0.3568
Epoch 11/20
17928/17928 [==============================] - 4s 197us/step - loss: 0.3567
Epoch